# Heating Mesh Tally on CAD geometry made from Shapes

This constructs a reactor geometry from 3 Shape objects each made from points.

The Shapes made include a breeder blanket, PF coil and a central column shield.

2D and 3D Meshes tally are then simulated to show nuclear heating, flux and tritium_production across the model.

This makes a 3D geometry and material for the centre column

In [1]:
import paramak

center_column = paramak.RotateMixedShape(
    points=[
        (50, 600, 'straight'),
        (150, 600, 'spline'),
        (100, 0, 'spline'),
        (150, -600, 'straight'),
        (50, -600, 'straight')
    ],
    name='center_column'
)

This makes a 3D geometry and material for breeder blanket. The azimuth_placement_angle argument is used to repeat the geometry around the Z axis at specified angles.

In [2]:
blanket = paramak.RotateSplineShape(
    points=[
        (600, 0),
        (600, -20),
        (500, -300),
        (400, -300),
        (400, 0),
        (400, 300),
        (500, 300),
        (600, 20)
    ],
    name='blanket',
    rotation_angle=40,
    azimuth_placement_angle=[0, 45, 90, 135, 180, 225, 270, 315],
)

This makes a reactor object from the three components

In [3]:
my_reactor = paramak.Reactor([blanket, center_column])

my_reactor.show()

Overwriting auto display for cadquery Workplane and Shape
100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.18s


CadViewerWidget(anchor=None, cad_width=800, glass=False, height=600, pinning=False, theme='light', title=None,…

This section forms the neutronics model by combining the 3D model, the plasma source and some assigned materials. Additionally, the tallies to record the heating are specified.

In [4]:
my_reactor.export_dagmc_h5m(filename='dagmc.h5m')

['blanket', 'center_column']
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Circle)
Info    : [ 10%] Meshing curve 2 (Circle)
Info    : [ 10%] Meshing curve 3 (Circle)
Info    : [ 10%] Meshing curve 4 (BSpline)
Info    : [ 20%] Meshing curve 5 (Circle)
Info    : [ 20%] Meshing curve 6 (Line)
Info    : [ 20%] Meshing curve 7 (Circle)
Info    : [ 30%] Meshing curve 8 (BSpline)
Info    : [ 30%] Meshing curve 9 (BSpline)
Info    : [ 30%] Meshing curve 10 (Circle)
Info    : [ 40%] Meshing curve 11 (BSpline)
Info    : [ 40%] Meshing curve 12 (BSpline)
Info    : [ 40%] Meshing curve 13 (Circle)
Info    : [ 50%] Meshing curve 14 (BSpline)
Info    : [ 50%] Meshing curve 15 (BSpline)
Info    : [ 50%] Meshing curve 16 (Circle)
Info    : [ 60%] Meshing curve 17 (BSpline)
Info    : [ 60%] Meshing curve 18 (BSpline)
Info    : [ 60%] Meshing curve 19 (Circle)
Info    : [ 70%] Meshing curve 20 (BSpline)
Info    : [ 70%] Meshing curve 21 (BSpline)
Info    : [ 70%] Meshing curve 22 (Circle)
I

'dagmc.h5m'

This next section combines the geometry with the materials and specifies a few mesh tallies

In [5]:
# makes use of the previously created neutronics geometry (h5m file) and assigns
# actual materials to the material tags. 

import openmc_dagmc_wrapper as odw

import neutronics_material_maker as nmm

# this links the material tags in the dagmc h5m file with materials.
# these materials are input as strings so they will be looked up in the
# neutronics material maker package
material_tag_to_material_dict = {
    'blanket': nmm.Material.from_library(name='DT_plasma'),
    'center_column': nmm.Material.from_library(name='Li4SiO4'),
}

geometry = odw.Geometry(h5m_filename='dagmc.h5m')

materials = odw.Materials(
    h5m_filename='dagmc.h5m',
    correspondence_dict=material_tag_to_material_dict
)

This section sets the the neutronics results to record (know as tallies).

In [7]:
from dagmc_bounding_box import DagmcBoundingBox
import openmc

# finds bounding box size from the geometry size
corners = DagmcBoundingBox('dagmc.h5m').corners()

mesh = openmc.RegularMesh()
mesh.dimension = [100, 100, 100]  # number of mesh cells in each dimension (resolution)

mesh.lower_left = corners[0] # x,y,z coordinates
mesh.upper_right = corners[0] # x,y,z coordinates

mesh_filter = openmc.MeshFilter(mesh)

# Create flux mesh tally to score alpha production
mesh_tally = openmc.Tally(tally_id=1, name='alpha_production_on_mesh')  # note the tally_id is specified
mesh_tally.filters = [mesh_filter]
mesh_tally.scores = ['(n,Xa)']  # where X is a wild card


tallies = openmc.Tallies([mesh_tally])

/opt/conda/lib/python3.8/site-packages/openmc/mixin.py:67: IDWarning: Another Tally instance already exists with id=1.
  warn(msg, IDWarning)


Sets simulation intensity, combines the geometry, materials, tallies and settings into a single object and runs the simulation

In [8]:
import openmc_plasma_source as ops

settings = openmc.Settings()
settings.inactive = 0
settings.run_mode = 'fixed source'
settings.batches = 4
settings.particles = 1000

# assigns a ring source of DT energy neutrons to the source using the
# openmc_plasma_source package, more details here
# https://github.com/fusion-energy/openmc-plasma-source/
settings.source = ops.FusionRingSource(fuel="DT", radius=350)


my_model = openmc.Model(
    materials=materials,
    geometry=geometry,
    settings=settings,
    tallies=tallies
)
!rm *.h5  # just removes old summary.h5 or statepoint files
statepoint_file = my_model.run()

rm: cannot remove '*.h5': No such file or directory
                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%

The following section opens the statepoint and converts the tally result into a VTK file. VTK files can be opened in paraview

In [ ]:

# assumes you have a statepoint file from the OpenMC simulation
statepoint = openmc.StatePoint('statepoint.4.h5')

# this shows the tallies present in the statepoint file
print(statepoint.tallies)

# loads up a tally from the statepoint using it's name
my_tally = statepoint.get_tally(name='alpha_production_on_mesh')

# converts the tally result into a VTK file
mesh.write_data_to_vtk(
    datasets={'mean': my_tally.mean},
    filename = "vtk_file_from_openmc_mesh.vtk",
)

Now download the VTK file by moving to the file explorer, right mouse clicking on the file and selecting Download.

Open the file in Paraview or another VTK file viewer on your local computer